In [58]:
from ultralytics import YOLO
model = YOLO("modelFolder/model_v4.pt")
results = model(source="w640h360/9_Color.png",device="cpu", project = "results",name="test", conf=0.8 )
for result in results:
    image_points = result.masks.xyn
    print(len(image_points))
    


image 1/1 c:\Team_Project-1\w640h360\9_Color.png: 256x416 2 boxs, 94.7ms
Speed: 2.0ms preprocess, 94.7ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 416)


2


In [59]:
from crop_ply import *
xyz_output= cropPly(image_points,"w640h360/9.ply")
print(xyz_output)

[PointCloud with 925 points., PointCloud with 2290 points.]


In [60]:
import open3d as o3d
import numpy as np

# 포인트 클라우드를 시각화
for cloud in xyz_output:
    # 포인트 클라우드의 경계 상자 계산
    aabb = cloud.get_axis_aligned_bounding_box()
    aabb.color = (1, 0, 0)

    obb = cloud.get_oriented_bounding_box()
    obb.color = (0, 1, 0)

    # 클라우드 포인트의 중심 위치 계산
    center = np.mean(np.asarray(cloud.points), axis=0)
    print(center)

    # 중앙 위치에 좌표 프레임 메시 생성
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.07)  # size를 조절하여 좌표 프레임 크기 설정 가능

    # 중앙 위치에 좌표 프레임 메시를 이동
    mesh.translate(center)

    # 시각화
    o3d.visualization.draw_geometries([cloud, aabb, obb, mesh])

    # PCA를 수행하여 주요 주성분 벡터를 찾습니다.
    points = np.asarray(cloud.points)
    covariance_matrix = np.cov(points, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)

    # 주요 주성분 벡터와 각각의 축과의 각도를 계산합니다.
    x_axis = np.array([1, 0, 0])  # x 축
    y_axis = np.array([0, 1, 0])  # y 축
    z_axis = np.array([0, 0, 1])  # z 축

    angle_x_rad = np.arccos(np.dot(eigenvectors[:, -1], x_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(x_axis)))
    angle_x_deg = np.degrees(angle_x_rad)

    angle_y_rad = np.arccos(np.dot(eigenvectors[:, -1], y_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(y_axis)))
    angle_y_deg = np.degrees(angle_y_rad)

    angle_z_rad = np.arccos(np.dot(eigenvectors[:, -1], z_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(z_axis)))
    angle_z_deg = np.degrees(angle_z_rad)

    print(f"요피치 롤(x 축 기준): {angle_x_deg}도")
    print(f"요피치 롤(y 축 기준): {angle_y_deg}도")
    print(f"요피치 롤(z 축 기준): {angle_z_deg}도")


[    0.57881     0.60836    -0.56029]
요피치 롤(x 축 기준): 127.04044031040468도
요피치 롤(y 축 기준): 141.74192506594449도
요피치 롤(z 축 기준): 98.242794340815도
[    0.56657     0.35753      -0.528]
요피치 롤(x 축 기준): 95.12756651815148도
요피치 롤(y 축 기준): 164.2167922625052도
요피치 롤(z 축 기준): 75.11405530608883도
